# Visualise detection results

In [1]:
import os
import napari
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask.array.image import imread as imr
from copy import deepcopy

In [2]:
import tkinter as tk
from tkinter import filedialog

In [3]:
def choose_home_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askdirectory(initialdir= "/", title='Please select a directory')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path

def get_session_name():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askdirectory(initialdir= "/", title='Please select a directory')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path

In [4]:
homeFolder = choose_home_folder()
print("Chosen home folder:", homeFolder)
sessionNames = [i for i in os.listdir(homeFolder) if '_F' in i and os.path.isdir(os.path.join(homeFolder,i))]

print('List of movies folders found\n')
sessionNames

Chosen home folder: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE
List of movies folders found



['Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06']

In [6]:
i=0
sessionName = sessionNames[i]
imsQ = sessionName.split('/')[-1].split('_F')[1]


nameKey = sessionName.split('/')[-1].split('_F')[0]+'_F'


pathToTimeFrames = os.path.join(homeFolder,nameKey+imsQ+'/*'+imsQ+'*.tif')
# path_input = os.path.join(homeFolder,nameKey+imsQ+'/cell_'+str(cellNumber)+'/')

movieFolder = os.path.join(homeFolder,nameKey+imsQ+'/')

# 
# MaxTimePoint = np.max([len(os.listdir(os.path.join(homeFolder,sessionName,i))) for i in os.listdir(movieFolder) if "cell_" in i])
timeStack = imr(pathToTimeFrames)
maxImage = np.max(timeStack, axis=1)
MaxTimePoint = maxImage.shape[0]
maxImage

dask.array<amax-aggregate, shape=(900, 1024, 1024), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>

In [7]:
sessionName

'Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06'

In [12]:


cellNumber = "11"


# Hela_h9_h2_k11_mcpsg_1hrbasal_14h_prostratin_exp2_F

nucleiStackForm = nameKey+imsQ+"_cell_"
pathToTimeSequenceFullView = os.path.join(homeFolder,nameKey+imsQ,'*.tif')
cell_Crop_Folder = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/'
pathToTimeSequenceCell = os.path.join(homeFolder,nameKey+imsQ,'cell_'+str(cellNumber),'*.tif')
moviePath = homeFolder+nameKey+imsQ

sequenceCell = imr(pathToTimeSequenceCell)
mipSequenceCell = np.max(sequenceCell, axis=1)
MaxTimePoint = sequenceCell.shape[0]


# spcl = np.load(cell_Crop_Folder+'cell_'+str(cellNumber)+'_spots_and_clusters_0.3.npz',allow_pickle=True)

# spotsFrame = spcl['spotsFrame']
# clustersFrames = spcl['clustersFrames']
# ThresholdFrames = spcl['ThresholdFrames']
# reference_spot = spcl['reference_spot']
# refSpot = deepcopy(reference_spot)

def getDetectedPointsForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-1])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:]
    return pts_coords

def getDetectedClustersForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-3])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:3]
    return pts_coords

def set_pts_features(pts_layer, cls_layer, pts_coordinates, cluster_coordinate, step): #TxLayer
    # step is a 4D coordinate with the current slider position for each dim
    frameNumber = step[0]  # grab the leading ("time") coordinate
    pts_layer.data = getDetectedPointsForFrame(pts_coordinates,frameNumber)
    cls_layer.data = getDetectedClustersForFrame(cluster_coordinate,frameNumber)


# pts_coordinates = spotsFrame
# cluster_coordinate = clustersFrames

viewer = napari.Viewer()
image_layer = viewer.add_image(
        mipSequenceCell, colormap='green' #maxImageCell
        )
# if image_layer.data.ndim == 4:
#     bigfishSpots = spotsFrame
# elif image_layer.data.ndim == 3:
#     bigfishSpots = getDetectedPointsForFrame(pts_coordinates,int(np.shape(mipSequenceCell)[0]/2))
    
# bigfish_Spots = viewer.add_points(
#         getDetectedPointsForFrame(pts_coordinates,int(np.shape(mipSequenceCell)[0]/2)-1),
#         face_color='#00000000',
#         size=4,
#         edge_width=0.3,
#         edge_width_is_relative=False,
#         edge_color='white',
#         name = 'bigFish Detected Spots'
#         )

# bigfish_clusters = viewer.add_points(
#         getDetectedClustersForFrame(cluster_coordinate,int(np.shape(mipSequenceCell)[0]/2)-1),
#         face_color='#00000000',
#         size=8,
#         edge_width=0.3,
#         edge_width_is_relative=False,
#         edge_color='red',
#         symbol='diamond',
#         name = 'bigFish Clusters'
#         )

# viewer.dims.events.current_step.connect(
#         lambda event: set_pts_features(bigfish_Spots, bigfish_clusters, pts_coordinates, cluster_coordinate, event.value)
#         )